In [1]:
# initialize all the necessary library
from asyncio import protocols
import pandas as pd
from pyhive import presto
import numpy as np
import os
import warnings
from scipy import stats

presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
presto_conn = presto.connect(
    host=presto_host,
    port=presto_port,
    username='dharmendra.k@rapido.bike',
    protocol='http',
    catalog='hive',
)


### define functions to pull metrics/data

In [2]:

def get_start_end_date_from_week(week_num, year):

    yyyy_wk_num = str(year) + '-W' + str(week_num)
    # -1 and - % w pattern tells the parser to pick the Monday in that week
    wk_start = pd.to_datetime(yyyy_wk_num + '-1', format="%Y-W%W-%w")
    wk_end = wk_start + pd.to_timedelta('6 D')

    return wk_start.strftime('%Y-%m-%d'), wk_end.strftime('%Y-%m-%d')


def get_captain_completed_rides(single_date, city, conn_details):

    query = """
        select 
            city_name, order_date,
            captain_id,
            count(order_id) as dropped_order,
            count(case when hhmmss >= '070000' and hhmmss < '120000' then order_id end) as morning_order,
            count(case when hhmmss >= '120000' and hhmmss < '170000' then order_id end) as afternoon_order,
            count(case when hhmmss >= '170000' and hhmmss <= '235959' then order_id end) as evening_night_order,
            count(case when hhmmss >= '000000' and hhmmss < '070000' then order_id end) as mid_night_order
        from orders.order_logs_snapshot
        where yyyymmdd = date_format(date('{start_date}'), '%Y%m%d')
        and city_name = '{city}'
        and service_obj_service_name = 'Link'
        and order_status = 'dropped'
        and (spd_fraud_flag = false or spd_fraud_flag is null)
        group by 1, 2, 3
    """.format(start_date=single_date, city=city)

    captain_agg_order_df = pd.read_sql_query(query, conn_details)

    return captain_agg_order_df



def get_daily_weekly_incentive_details(start_date, end_date, city, conn_details):

    query = """
        with 
            incentive_tbl as(
                select
                    incentive_id, 
                    --cities,
                    incentivetype,
                    startdate, enddate
                from(
                    select
                        _id as incentive_id, 
                        --cities[1] as city,
                        incentivetype,
                        startdate, enddate,
                        row_number() over(partition by _id order by updated_epoch desc) as row
                    from raw.mongodb_rapidopayroll_incentives_immutable
                    where yyyymmdd >= date_format(date_trunc('week', date('{start_date}')) - interval '2' day, '%Y%m%d')
                    and yyyymmdd <= date_format(date('{end_date}'), '%Y%m%d')
                    and json_array_contains(cities, '{city_name}')
                    and json_array_contains(servicenames, 'Link')
                    and incentivetype in ('Daily', 'Weekly Fixed')
                    )
                where row=1
                and startdate >= '{start_date}'
                and enddate <= '{end_date}'
            )
            
        select 
            yyyymmdd, startdate, riderid,
            sum(case when incentivetype = 'Daily' then incentive_amt end) as daily_incentive_amt,
            sum(case when incentivetype = 'Weekly Fixed' then incentive_amt end) as weekly_incentive_amt
        from(
            select 
                riderid, yyyymmdd,
                incentive_id, incentive_amt
            from(
                select riderid, yyyymmdd,
                    tincentiveIdl as incentive_id,
                    cast(amount as double) as incentive_amt,
                    row_number() over(partition by _id order by updated_epoch desc) as row
                from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                where yyyymmdd >= date_format(date('{start_date}'), '%Y%m%d')
                and yyyymmdd <= date_format(date('{end_date}') + interval '1' day, '%Y%m%d')
                and transactiontype = 'specialIncentive'
                and status = 'success' 
                and city = '{city_name}'
                )
            where row = 1
            ) as rpn_tbl
        inner join incentive_tbl on rpn_tbl.incentive_id = incentive_tbl.incentive_id
        group by 1, 2, 3
    """.format(start_date=start_date, end_date=end_date, city_name=city)

    print(query)

    incentive_df = pd.read_sql_query(query, conn_details)

    return incentive_df


def get_captain_daily_login_hrs(date, city, conn_details):

    query = """
        SELECT
            userid as captain_id,
            date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') AS logindate,
            cast(sum(duration) as double) / 3600000.0 as login_hours,
            SUM(
                CASE WHEN quarter_hour >= '0600' and quarter_hour <= '1159'
                then cast(duration as double) / 3600000.0 ELSE 0 END
            ) AS morning_duration,
            SUM(
                CASE WHEN quarter_hour >= '1200' and quarter_hour <= '1759'
                then cast(duration as double) / 3600000.0 ELSE 0 END
            ) AS afternoon_duration,
            SUM(
                CASE WHEN quarter_hour >= '1800' and quarter_hour <= '2359'
                then cast(duration as double) / 3600000.0 ELSE 0 END
            ) AS evening_duration
        from hive.datasets.captain_login_hours
        where yyyymmdd = date_format(date('{start_date}'), '%Y%m%d')
        and status in ('2', '3', '6', '7', '8', '10')
        and userid in (
            select captainId from datasets.captain_single_view
            where registeredcity=lower('{city}')
            and activationdate is not null
        )
        GROUP BY 1, 2
    """.format(start_date=date, city=city)

    # print(query)

    login_hrs_df = pd.read_sql_query(query, conn_details)

    return login_hrs_df


def get_incentive_performance(start_date, end_date, city, presto_conn):

    query = """
        with
            city_tbl as(
                select displayname as city
                from legacy.cities
                where 1=1
                and displayname = '{city}'
            )
            ,

            incentive_creation_tbl as(
                select
                    incentivetype, incentivename, incentive_id, cities, servicenames, startdate, enddate, uuid,
                    transform(time_slot, x -> cast(json_extract(x, '$.fromTime') as varchar)) as fromTime,
                    transform(time_slot, x -> cast(json_extract(x, '$.toTime') as varchar)) as toTime,
                    transform(rules, x -> cast(json_extract(x, '$.index') as double)) as slabs,
                    transform(rules, x -> cast(json_extract(x, '$.order') as double)) as slab_order_target,
                    transform(rules, x -> cast(json_extract(x, '$.amount') as double)) as slab_amount
                from
                    (
                    select incentivename, incentive_id, cities, servicenames, incentivetype,
                    startdate, enddate,
                    cast(json_extract(goals, '$.rules') as array<json>) as rules,
                    cast(json_extract(goals, '$.timeSlot') as array<json>) as time_slot,
                    cast(json_extract(goals, '$.uuid') as varchar) as uuid
                    from(
                        select
                            incentivename, _id as incentive_id, cities, servicenames, incentivetype,
                            startdate, enddate,
                            element_at(map_values(cast(cast(goals as json) as map(varchar, array(json)))), 1)[1] as goals,
                            row_number() over(partition by _id order by updated_epoch) as row
                        from raw.mongodb_rapidopayroll_incentives_immutable
                        where yyyymmdd >= date_format(date_trunc('week', date('{start_date}')) - interval '2' day, '%Y%m%d')
                        and yyyymmdd <= date_format(date('{end_date}'), '%Y%m%d')
                        and startdate >= date_format(date('{start_date}'), '%Y-%m-%d')
                        and enddate <= date_format(date('{end_date}'), '%Y-%m-%d')
                        and json_array_contains(cities, (select city from city_tbl))
                        -- and incentivetype = 'Weekly Fixed'
                        and (incentivetype = 'Daily' or incentivetype = 'Weekly Fixed')
                        )
                    where row = 1
                )
            )

        -- select
        --    riderid, startdate, enddate,
        --    slab_order_target,
        --    slab_amount, is_set_started,
        --    is_rule_started, is_rule_completed
        --    order_target, order_amount,
        --    rule_started, rule_completed
        --from(
            select
                incentivetype, incentivename, incentiveid,
                riderid, startdate, enddate,
                fromTime, toTime,
                slab_order_target, slab_amount, is_set_started,
                uuid, transform(rules_status, x-> cast(json_extract(x, '$.isRuleStarted') as boolean)) as is_rule_started,
                transform(rules_status, x-> cast(json_extract(x, '$.isRuleCompleted') as boolean)) as is_rule_completed
            from(
                select
                    incentivetype, incentivename, incentiveid, riderid,
                    startdate, enddate,
                    slab_order_target, slab_amount,
                    fromTime, toTime,
                    transform(field_status, x -> json_extract(x, '$.isSetStarted')) as is_set_started,
                    transform(field_status, x -> json_extract(x, '$.uuid')) as uuid,
                    cast(cast(transform(field_status, x -> json_extract(x, '$.rulesStatus')) as array<json>)[1] as array<json>) as rules_status
                from(
                    select
                        incentivetype, incentivename, incentiveid, riderid,
                        slab_order_target, slab_amount,
                        startdate, enddate,
                        fromTime, toTime,
                        cast(
                            transform(cast(incentiveProgress_json as array<json>),
                            x-> json_extract(x, '$.fieldStatus'))[1] as array<json>
                            ) as field_status
                    from raw.mongodb_rapidopayroll_incentive_riders_snapshot as incentive_progress_tbl
                    inner join incentive_creation_tbl on incentive_progress_tbl.incentiveid = incentive_creation_tbl.incentive_id
                    where yyyymmdd >= date_format(date('{start_date}') - interval '1' day, '%Y%m%d')
                    and yyyymmdd <= date_format(date('{end_date}') + interval '1' day, '%Y%m%d')
                )
            )
        --)
        -- cross join unnest(slab_order_target, slab_amount, is_rule_started, is_rule_completed) as tbl1(order_target, order_amount, rule_started, rule_completed)
    """.format(start_date=start_date, end_date=end_date, city=city)

    rider_incentive_df = pd.read_sql_query(query, presto_conn)

    return(rider_incentive_df)



# def get_rider_

login_hrs_state = dict(
    zip(
        ['2', '3', '6', '7', '8', '10'],
        ['ready_for_ride', 'order_receiving',
         'on_the_way', 'arrived', 'started', 'reached']
    )
)

login_hrs_state


{'2': 'ready_for_ride',
 '3': 'order_receiving',
 '6': 'on_the_way',
 '7': 'arrived',
 '8': 'started',
 '10': 'reached'}

### config

In [3]:

city = 'Mumbai'
week_num = 23
year = 2022

incentive_type = 'Daily'

start_date, end_date = get_start_end_date_from_week(week_num, year)
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

daily_list = [
    start_date + pd.to_timedelta('{} D'.format(val))
    for val in range(0, (end_date - start_date).days + 1, 1)
]
daily_list = [date.strftime('%Y-%m-%d') for date in daily_list]

perf_wk = '{:02d}'.format(week_num)
start_date = [start_date.strftime('%Y-%m-%d')]
end_date = [end_date.strftime('%Y-%m-%d')]


print('City \t\t\t:', city)
print('Week Start Date \t:', start_date)
print('Week End Date: \t:', end_date)
print('Date List \t\t:', daily_list)


rides_file_name = '{city}_captain_rides_df_wk_{week_num}.csv'.format(
    city=city, week_num=perf_wk
)

login_hrs_file_name = '{city}_captain_login_hrs_df_wk_{week_num}.csv'.format(
    city=city, week_num=perf_wk
)

incentive_file_name = '{city}_captain_incentive_df_wk_{week_num}.csv'.format(
    city=city, week_num=perf_wk
)

incentive_progress_file_name = (
    "{city}_{incentive_type}_incentive_progress_df_wk_{week_num}.pkl"
    .format(
        city=city, incentive_type=incentive_type,
        week_num='{:02}'.format(week_num)
    )
)

print(
    'required dataset :\n',
    rides_file_name,
    login_hrs_file_name,
    incentive_file_name,
    incentive_progress_file_name,
    sep='\n'

)


City 			: Mumbai
Week Start Date 	: ['2022-06-06']
Week End Date: 	: ['2022-06-12']
Date List 		: ['2022-06-06', '2022-06-07', '2022-06-08', '2022-06-09', '2022-06-10', '2022-06-11', '2022-06-12']
required dataset :

Mumbai_captain_rides_df_wk_23.csv
Mumbai_captain_login_hrs_df_wk_23.csv
Mumbai_captain_incentive_df_wk_23.csv
Mumbai_Daily_incentive_progress_df_wk_23.pkl


### pull data and store it locally

In [4]:

if not(os.path.exists(rides_file_name)):
    captain_rides_df = pd.DataFrame()
    for date in daily_list:
        temp_rides_df = get_captain_completed_rides(date, city, presto_conn)
        captain_rides_df = pd.concat([captain_rides_df, temp_rides_df])
        print(
            "{city} Fetched captain agg daily rides as of {date} and total captains is {count}"
            .format(city=city, date=date, count=temp_rides_df.shape[0])
        )

    captain_rides_df.to_csv(
        rides_file_name,
        index=False
    )
    print(captain_rides_df.head(), '\n')
else:
    # read the captain daily rides dataframe from local
    captain_rides_df = pd.read_csv(rides_file_name)
    print(captain_rides_df.head(), '\n')


if not(os.path.exists(login_hrs_file_name)):

    captain_login_hrs_df = pd.DataFrame()

    for date in daily_list:
        temp_login_hrs_df = get_captain_daily_login_hrs(
            date, city, presto_conn)
        captain_login_hrs_df = pd.concat(
            [captain_login_hrs_df, temp_login_hrs_df])
        print('{} Fetched captain login hours for {} and total captains is {}'.format(
            city, date, temp_login_hrs_df.shape[0]))

    print('captain login hours dataframe to be saved as ', login_hrs_file_name)

    captain_login_hrs_df.to_csv(
        login_hrs_file_name,
        index=False
    )
    print(captain_login_hrs_df.head(), '\n')
else:
    captain_login_hrs_df = pd.read_csv(login_hrs_file_name)
    print(captain_login_hrs_df.head(), '\n')


if not(os.path.exists(incentive_file_name)):
    incentive_amt_df = pd.DataFrame()
    for start, end in zip(start_date, end_date):
        temp_incentive_df = get_daily_weekly_incentive_details(
            start, end, city, presto_conn)
        incentive_amt_df = pd.concat([incentive_amt_df, temp_incentive_df])
        print("{city} Fetched captain incentive amount for the period {start} to {end}".format(
            city=city, start=start, end=end))

    incentive_amt_df.to_csv(
        incentive_file_name,
        index=False
    )
    print(incentive_amt_df.head(), '\n')
else:
    # read the incentive achievement
    incentive_amt_df = pd.read_csv(incentive_file_name)
    print(incentive_amt_df.head(), '\n')

incentive_amt_df['yyyymmdd'] = pd.to_datetime(incentive_amt_df['yyyymmdd'], format='%Y%m%d')

if os.path.exists(incentive_progress_file_name):
    rider_incentive_progress_df = pd.read_pickle(incentive_progress_file_name)
    print(rider_incentive_progress_df.head())
else:
    print('Incentive Progress file does not exits for week {}'.format(perf_wk))

# incentive progress part
# if not (os.path.exists(incentive_progress_file_name)):

#     rider_incentive_progress_df = get_incentive_performance(
#         start_date[0], end_date[0], city, presto_conn
#     )
#     # save the file locally
#     rider_incentive_progress_df.to_pickle(incentive_progress_file_name)
#     print(rider_incentive_progress_df.head())
# else:
#     rider_incentive_progress_df = pd.read_pickle(incentive_progress_file_name)
#     print(rider_incentive_progress_df.head())


# rider_incentive_progress_df['slab'] = (
#     rider_incentive_progress_df['slab_order_target']
#     .apply(lambda x: list(range(1, len(x) + 1)))
# )


  city_name  order_date                captain_id  dropped_order  \
0    Mumbai  2022-06-06  612c7eb334b93de406358227              2   
1    Mumbai  2022-06-06  610a46c780f61d07762cc5d4             20   
2    Mumbai  2022-06-06  62971adfa81bc548e659e1f4             11   
3    Mumbai  2022-06-06  5e728b45f49633436e2fa906              6   
4    Mumbai  2022-06-06  6234b3da6b6b643417706f1b             10   

   morning_order  afternoon_order  evening_night_order  mid_night_order  
0              0                0                    0                2  
1              0                7                    8                5  
2              2                3                    3                3  
3              0                4                    2                0  
4              0                4                    5                1   

                 captain_id   logindate  login_hours  morning_duration  \
0  615d3aa72e6fe8afb2b5eb27  2022-06-06     8.521111          0.458611 

In [318]:


# captain_rides_df = captain_rides_df[
#     captain_rides_df['order_date'] != '2022-06-08'
# ]
# captain_login_hrs_df = captain_login_hrs_df[
#     captain_login_hrs_df['logindate'] != '2022-06-08'
# ]


# for date in daily_list[2:3]:
#     temp_rides_df = get_captain_completed_rides(date, city, presto_conn)
#     captain_rides_df = pd.concat([captain_rides_df, temp_rides_df])
#     print(
#         "{city} Fetched captain agg daily rides as of {date} and total captains is {count}"
#         .format(city=city, date=date, count=temp_rides_df.shape[0])
#     )

#     captain_rides_df.to_csv(
#         rides_file_name,
#         index=False
#     )
#     print(captain_rides_df.head(), '\n')

# for date in daily_list[2:3]:
#     temp_login_hrs_df = get_captain_daily_login_hrs(
#         date, city, presto_conn)
#     captain_login_hrs_df = pd.concat(
#         [captain_login_hrs_df, temp_login_hrs_df])
#     print('{} Fetched captain login hours for {} and total captains is {}'.format(
#         city, date, temp_login_hrs_df.shape[0]))

#     print('captain login hours dataframe to be saved as ', login_hrs_file_name)

#     captain_login_hrs_df.to_csv(
#         login_hrs_file_name,
#         index=False
#     )
#     print(captain_login_hrs_df.head(), '\n')


### Function Defined for Analysis 

In [5]:
def compute_weekly_captain_metrics(rides_df, login_df, incentive_df,
                                     rides_rider_field, rides_date_field,
                                     login_rider_field, login_date_field,
                                     incentive_rider_field, incentive_date_field
                                     ):

    """
    :param rides_df: a dataframe for captain daily link completed rides
    :param login_df: a dataframe for captain daily login hours on the platform
    :param incentive_df: a dataframe capturing daily and weekly incentive at catpain and day level
    :param rides_rider_field: captain id field name in rides df
    :param rides_date_field: date field name in rides df
    :param login_rider_field: captain id field name in login df
    :param login_date_field: date field name in login df
    :param incentive_rider_field: captain id field name in incentive df
    :param incentive_date_field: date field name in incentive df
    :param return: aggregated df at captain and week

    """

    rides_df['date'] = pd.to_datetime(rides_df[rides_date_field])
    rides_df['year'] = (rides_df['date'].dt.isocalendar().year)
    rides_df['year'] = rides_df['year'].astype('str')
    rides_df['week'] = (rides_df['date'].dt.isocalendar().week)
    rides_df['week'] = rides_df['week'].map('{:02}'.format)
    rides_df['week_num'] = (
        rides_df['year'] +
        '-' + 
        rides_df['week'].astype('str') 
    )

    working_days_df = (
        rides_df[[rides_rider_field, rides_date_field, 'week_num']]
        .groupby([rides_rider_field, 'week_num'])
        .agg(
            net_active_days=(rides_date_field, 'nunique')
        )
        .reset_index()
    )

    rides_df = (
        rides_df
        .groupby([rides_rider_field, 'week_num'])
        .sum()
        .reset_index()
    )

    login_df['date'] = pd.to_datetime(login_df[login_date_field])
    login_df['year'] = login_df['date'].dt.isocalendar().year
    login_df['year'] = login_df['year'].astype('str')
    login_df['week'] = (login_df['date'].dt.isocalendar().week)
    login_df['week'] = login_df['week'].map('{:02}'.format)
    login_df['week_num'] = (
        login_df['year'] +
        '-' +
        login_df['week'].astype('str')
    )

    login_df = (
        login_df
        .groupby([login_rider_field, 'week_num'])
        .sum()
        .reset_index()
    )

    incentive_df['date'] = pd.to_datetime(incentive_df[incentive_date_field])
    incentive_df['year'] = incentive_df['date'].dt.isocalendar().year
    incentive_df['year'] = incentive_df['year'].astype('str')
    incentive_df['week'] = (incentive_df['date'].dt.isocalendar().week)
    incentive_df['week'] = incentive_df['week'].map('{:02}'.format)
    incentive_df['week_num'] = (
        incentive_df['year'] +
        '-' +
        incentive_df['week'].astype('str')
    )

    incentive_df = (
        incentive_df
        .groupby([incentive_rider_field, 'week_num'])
        .sum()
        .reset_index()
    )

    final_df = (
        rides_df
        .merge(
            login_df,
            left_on=[rides_rider_field, 'week_num'],
            right_on=[login_rider_field, 'week_num'],
            how='left',
            suffixes=(('', '_right'))
        )
        .merge(
            incentive_df,
            left_on=[rides_rider_field, 'week_num'],
            right_on=[incentive_rider_field, 'week_num'],
            how='left',
            suffixes=(('', '_right'))
        )
        .merge(
            working_days_df,
            on=[rides_rider_field, 'week_num'],
            how='left'
        )
    )
    
    return final_df


def get_wow_retention(data, rider_col_name, date_col_name):
    """
    :param data: requires pandas dataframe with rider and ride date as the column
    :param return: week on week retention dataframe
    """

    data['year'] = (
        pd.to_datetime(data[date_col_name])
        .dt.isocalendar().year
    )

    data['week'] = (
        pd.to_datetime(data[date_col_name])
        .dt.isocalendar().week
    )

    data['week'] = data.week.map("{:02}".format)

    data['week'] = (
        data['year'].astype('str') +
        '-' +
        data['week'].astype('str')
    )

    weekly_agg_df = (
        data[[rider_col_name, 'week']]
        .groupby([rider_col_name, 'week'])
        .count()
        .reset_index()
    )

    week_list = weekly_agg_df['week'].unique()
    week_list.sort()
    temp_df = pd.DataFrame()

    for week_one, week_two in zip(week_list[:-1], week_list[1:]):

        week_n1_df = (
            weekly_agg_df[
                weekly_agg_df['week'] == week_one
            ]
            .rename(
                columns={
                    'week': 'week_n1',
                    rider_col_name: 'rider_wk_n1'
                }
            )
        )

        week_n2_df = (
            weekly_agg_df[
                weekly_agg_df['week'] == week_two
            ]
            .rename(
                columns={
                    'week': 'week_n2',
                    rider_col_name: 'rider_wk_n2'
                }
            )
        )

        wow_df = (
            pd.merge(
                left=week_n1_df,
                right=week_n2_df,
                left_on='rider_wk_n1',
                right_on='rider_wk_n2',
                how='left'
            )
        )

        temp_df = pd.concat([temp_df, wow_df])

    retention_view_df = (
        temp_df
        .groupby(['week_n1'])
        .agg(
            week_in=('rider_wk_n1', 'nunique'),
            week_next=('rider_wk_n2', 'nunique')
        )
        .reset_index()
        .rename(columns={'week_n1': 'week'})
    )

    return retention_view_df


def get_captain_daily_level_metrics(rides_df, login_df, incentive_df,
                                     selectors_df, rides_rider_field, rides_date_field,
                                     login_rider_field, login_date_field, incentive_rider_field,
                                     incentive_date_field, selector_rider_field
                                     ):

    """
    :param rides_df: a dataframe for captain daily link completed rides
    :param login_df: a dataframe for captain daily login hours on the platform
    :param incentive_df: a dataframe capturing daily and weekly incentive at catpain and day level
    :param rides_rider_field: captain id field name in rides df
    :param rides_date_field: date field name in rides df
    :param login_rider_field: captain id field name in login df
    :param login_date_field: date field name in login df
    :param incentive_rider_field: captain id field name in incentive df
    :param incentive_date_field: date field name in incentive df
    :param return: aggregated df at captain and week

    """

    rides_df['date'] = pd.to_datetime(rides_df[rides_date_field])
    login_df['date'] = pd.to_datetime(login_df[login_date_field])
    incentive_df['date'] = pd.to_datetime(incentive_df[incentive_date_field])

    final_df = (
        rides_df
        .merge(
            login_df,
            left_on=[rides_rider_field, 'date'],
            right_on=[login_rider_field, 'date'],
            how='left',
            suffixes=(('', '_right'))
        )
        .merge(
            incentive_df,
            left_on=[rides_rider_field, 'date'],
            right_on=[incentive_rider_field, 'date'],
            how='left',
            suffixes=(('', '_right'))
        )
    )
    print(final_df.head())

    final_df['year'] = (final_df['date'].dt.isocalendar().year)
    final_df['year'] = final_df['year'].astype('str')
    final_df['week'] = (final_df['date'].dt.isocalendar().week)
    final_df['week'] = final_df['week'].map('{:02}'.format)
    final_df['week_num'] = (
        final_df['year'] + '-' +
        final_df['week'].astype('str')
    )
    
    final_df = (
        final_df
        .merge(
            selectors_df,
            left_on=[rides_rider_field],
            right_on=[selector_rider_field],
            how='left',
            suffixes=('', '_right')
        )
    )

    return final_df


def compute_aggregated_metrics(
        captain_daily_agg_df, grouping_var,
        agg_fields, agg_funcs):

    agg_df = (
        captain_daily_agg_df
        .groupby(grouping_var)
        .agg(
            dict(zip(agg_fields, agg_funcs))
        )
        .reset_index()
    )

    return agg_df


### metric computation

In [6]:
# read last week
experiment_name = 'experiment_selector'
perf_wk = '23'
experiment_selectors_file = (
    '{city}_{experiment_name}_wk_{week_num}.csv'
    .format(city=city, experiment_name=experiment_name, week_num=perf_wk)
)
experiment_selectors_file


'Mumbai_experiment_selector_wk_23.csv'

In [7]:
old_new_segment_match_df = pd.read_csv('poc_old_new_segment_mapping.csv')
segment_col_name = 'poc_segment'
old_new_segment_match_df.head()
old_new_segment_match_df.rename(
    columns={'segment': segment_col_name},
    inplace=True
)

print("reading last week {city} selector ...".format(city=city))
selectors_df = pd.read_csv(experiment_selectors_file)
print(
    '*' * 75, '\n',
    'Week {} selectors view: \n'.format(perf_wk),
    selectors_df.head(), '\n'
)

selectors_df = selectors_df.merge(
    old_new_segment_match_df,
    on=[segment_col_name],
    how='left'
)

selectors_df[segment_col_name] = selectors_df['req_segment']


reading last week Mumbai selector ...
*************************************************************************** 
 Week 23 selectors view: 
                       rider   poc_segment  mobilenumber    experiment_group
0  62295b72bbd06007fae57e26  UHP_Intra_HO    9930771525  control_ride_group
1  61bddec34fc413806e358867  UHP_Intra_HO    7507505554    test_poc_segment
2  620629e66e47ef4c20f4589a   MP_Inter_LO    9607238212  control_ride_group
3  62380ef5a63f41cf8cd845d8   MP_Inter_LO    7571020664  control_ride_group
4  61cc11929ba372650a2db19c   MP_Inter_LO    8082091734  control_ride_group 



### [weekly aggregated view]

In [107]:

# experiment_selectors_file = (
#     '{city}_selectors_wk_{week_num}.csv'
#     .format(city=city, week_num=perf_wk)
# )

captain_weekly_agg_df = compute_weekly_captain_metrics(
    rides_df=captain_rides_df,
    login_df=captain_login_hrs_df,
    incentive_df=incentive_amt_df,
    rides_rider_field='captain_id',
    rides_date_field='order_date',
    login_rider_field='captain_id',
    login_date_field='logindate',
    incentive_rider_field='riderid',
    incentive_date_field='yyyymmdd'
)

captain_weekly_agg_df['daily_rpr'] =(
    captain_weekly_agg_df['dropped_order'] /
    captain_weekly_agg_df['net_active_days']
) 

print(
    '*' * 75, '\n',
    'Week {} captain level aggregate metric: \n'.format(perf_wk),
    captain_weekly_agg_df.head()
)



*************************************************************************** 
 Week 23 captain level aggregate metric: 
                  captain_id week_num  dropped_order  morning_order  \
0  57dc4c0f724beb006ab37b0c  2022-23             13              0   
1  582b8985a9924a8572313ad7  2022-23              7              1   
2  5858a163417054710ef1e496  2022-23              4              4   
3  598cb1d2bf125c4459811cf9  2022-23              3              0   
4  59ad5bc75564bc9b1a5f9a0c  2022-23              3              1   

   afternoon_order  evening_night_order  mid_night_order  login_hours  \
0                1                   12                0     6.261111   
1                0                    4                2    11.585556   
2                0                    0                0     2.541667   
3                0                    3                0     1.301944   
4                0                    2                0     3.441389   

   morning_duration 

In [108]:
# # selectors_df.head()
# def fetch_rider_info_from_svoc(city, presto_conn_details):
#     query = """
#             select 
#                 captainid as captain_id,
#                 mobilenumber, lastridedate, shift
#             from datasets.captain_single_view
#             where (lastridecity = '{city}' or registeredcity = lower('{city}'))
#         """.format(city=city)

#     svoc_df = pd.read_sql_query(query, presto_conn_details)

#     return svoc_df


# svoc_file_name = '{city}_svoc_df_wk_{week_num}.csv'.format(
#     city=city, week_num=perf_wk)

# if not(os.path.exists(svoc_file_name)):
#     svoc_df = fetch_rider_info_from_svoc(city, presto_conn)
#     svoc_df['mobilenumber'] = svoc_df['mobilenumber'].astype('str')
#     svoc_df.to_csv(svoc_file_name, index=False)
# else:
#     svoc_df = pd.read_csv(svoc_file_name)
#     svoc_df['mobilenumber'] = svoc_df['mobilenumber'].astype('str')


In [109]:

def get_experiment_performance(weekly_agg_df, weekly_agg_rider_field, selector_df,
 selector_rider_field, experiment_group_name, segment_col_name):
    """
    :param weekly_agg_df: captain level weekly aggregated dataframe
    :param selector_df: a df having captain id with their cohort and segment
    """

    req_fields = [
        'rider', 'week_num', 'dropped_order', 'login_hours',
        'morning_duration', 'afternoon_duration', 'evening_duration',
        'daily_incentive_amt', 'weekly_incentive_amt', 'daily_rpr'
        # 'simplified_segment', 'cohort',
    ]

    experiment_analysis_df = (
        weekly_agg_df
        .merge(
            selector_df,
            left_on=[weekly_agg_rider_field],
            right_on=[selector_rider_field],
            how='inner'
        )
    )
    
    # calculate the agg view at experiment group and segment group
    level_1_grouping_vars = ['week_num', experiment_group_name]
    level_1_agg_df = experiment_analysis_df.groupby(level_1_grouping_vars)
    level_1_selectors_size_df = (
        selector_df[level_1_grouping_vars[1:] + [selector_rider_field]]
        .groupby(level_1_grouping_vars[1:])
        .agg(
            {
                selector_rider_field: 'nunique',
            }
        )
        .reset_index()
    )

    level_1_analysis_agg_df = (
        level_1_agg_df[req_fields[:-1]].sum().reset_index()
        .merge(
            level_1_agg_df
            .agg(
                net_riders=(weekly_agg_rider_field, 'nunique'),
                median_active_days=('net_active_days', 'median'),
                daily_rpr=('daily_rpr', 'mean')
            )
            .reset_index(),
            on=level_1_grouping_vars,
            how='left'
        )
        .merge(
            level_1_selectors_size_df,
            on=level_1_grouping_vars[1:],
            how='left'
        )
    )

    # calculate the agg view at experiment group and segment group
    level_2_grouping_vars = [
        'week_num',experiment_group_name, segment_col_name
    ]
    level_2_agg_df = experiment_analysis_df.groupby(level_2_grouping_vars)
    level_2_selectors_size_df = (
        selector_df[level_2_grouping_vars[1:] + [selector_rider_field]]
        .groupby(level_2_grouping_vars[1:])
        .agg({selector_rider_field: 'nunique'})
        .reset_index()
    )

    level_2_analysis_agg_df = (
        level_2_agg_df[req_fields[:-1]].sum().reset_index()
        .merge(
            level_2_agg_df
            .agg(
                net_riders=('captain_id', 'nunique'),
                median_active_days=('net_active_days', 'median'),
                daily_rpr=('daily_rpr', 'mean')
            )
            .reset_index(),
            on=level_2_grouping_vars,
            how='left'
        )
        .merge(
            level_2_selectors_size_df,
            on=level_2_grouping_vars[1:],
            how='left'
        )
    )

    print(level_2_analysis_agg_df.head())

    analysis_agg_df = pd.concat(
        [
            level_2_analysis_agg_df,
            level_1_analysis_agg_df,
        ]
    )

    analysis_agg_df['total_incentive']=(
        analysis_agg_df['daily_incentive_amt'] + 
        analysis_agg_df['weekly_incentive_amt']
    )

    analysis_agg_df['rpr'] = (
        analysis_agg_df['dropped_order'] /
        analysis_agg_df['net_riders']
    )

    analysis_agg_df['burn_per_rider'] = (
        analysis_agg_df['total_incentive'] /
        analysis_agg_df['net_riders']
    )

    analysis_agg_df['weekly_burn_per_rider'] = (
        analysis_agg_df['weekly_incentive_amt'] /
        analysis_agg_df['net_riders']
    )

    analysis_agg_df['weekly_burn_per_rider'] = (
        analysis_agg_df['weekly_incentive_amt'] /
        analysis_agg_df['net_riders']
    )

    analysis_agg_df['daily_burn_per_rider'] = (
        analysis_agg_df['daily_incentive_amt'] /
        analysis_agg_df['net_riders']
    )

    analysis_agg_df['rph'] = (
        analysis_agg_df['dropped_order'] /
        analysis_agg_df['login_hours']
    )

    analysis_agg_df['daily_burn_per_ride'] = (
        analysis_agg_df['daily_incentive_amt'] /
        analysis_agg_df['dropped_order']
    )

    analysis_agg_df.reset_index(drop=True, inplace=True)

    rearrange_cols = [
        'week_num', segment_col_name, experiment_group_name,
         selector_rider_field, 'net_riders', 'dropped_order',  'rpr',
        'burn_per_rider', 'weekly_burn_per_rider',
        'daily_burn_per_rider', 'median_active_days', 'rph',
        'daily_rpr', 'daily_burn_per_ride', 'daily_incentive_amt',
        'weekly_incentive_amt'
    ]


    return analysis_agg_df[rearrange_cols]



In [110]:
analysis_agg_df = get_experiment_performance(
    weekly_agg_df=captain_weekly_agg_df,
    selector_df=selectors_df,
    weekly_agg_rider_field='captain_id',
    selector_rider_field='rider',
    experiment_group_name='experiment_group',
    segment_col_name=segment_col_name
)

analysis_agg_df.to_clipboard()
print(analysis_agg_df)


  week_num    experiment_group poc_segment  dropped_order  login_hours  \
0  2022-23  control_ride_group     HP_D_HO           2403  1796.699444   
1  2022-23  control_ride_group     HP_D_MO            205   131.834722   
2  2022-23  control_ride_group    HP_D_UHO           3387  2368.337222   
3  2022-23  control_ride_group     LP_D_HO            268   406.279444   
4  2022-23  control_ride_group     LP_D_LO            621   580.175833   

   morning_duration  afternoon_duration  evening_duration  \
0        363.510000          536.212778        777.637222   
1         50.460556           27.515278         50.948611   
2        430.431944          776.839167       1036.639722   
3         58.692778          138.060278        180.062222   
4        106.203611          143.544444        277.998056   

   daily_incentive_amt  weekly_incentive_amt  net_riders  median_active_days  \
0              19920.0                1130.0         190                 2.0   
1               2040.0      

### Daily Aggregated View

In [8]:

captain_daily_agg_df = get_captain_daily_level_metrics(
    rides_df=captain_rides_df,
    login_df=captain_login_hrs_df,
    incentive_df=incentive_amt_df,
    selectors_df=selectors_df,
    rides_rider_field='captain_id',
    rides_date_field='order_date',
    login_rider_field='captain_id',
    login_date_field='logindate',
    incentive_rider_field='riderid',
    incentive_date_field='yyyymmdd',
    selector_rider_field='rider'

)

print(
    '*' * 75, '\n',
    'Week {} captain level aggregate metric: \n'.format(perf_wk),
    captain_daily_agg_df.head()
)


rider_seg_exp_daily_df = captain_daily_agg_df.copy()
agg_fields = [
    'dropped_order', 'morning_order', 'afternoon_order',
    'evening_night_order', 'mid_night_order', 'morning_duration',
    'afternoon_duration', 'evening_duration', 'daily_incentive_amt',
    'weekly_incentive_amt',
]

agg_funcs = ['sum'] * len(agg_fields)

grouping_vars = [
    'city_name', 'week_num',
    'poc_segment', 'experiment_group'
]

daily_seg_metric_sum_df = compute_aggregated_metrics(
    captain_daily_agg_df=rider_seg_exp_daily_df,
    grouping_var=grouping_vars,
    agg_fields=agg_fields,
    agg_funcs=agg_funcs,
)


daily_incentive_df = compute_aggregated_metrics(
    captain_daily_agg_df=rider_seg_exp_daily_df,
    grouping_var=['city_name', 'week_num', 'order_date',
                  'poc_segment', 'experiment_group'],
    agg_fields=['daily_incentive_amt', 'dropped_order', 'captain_id'],
    agg_funcs=['sum', 'sum', 'nunique'],
)

daily_incentive_df['daily_burn_per_rider'] =(
    daily_incentive_df['daily_incentive_amt'].astype('int') /
    daily_incentive_df['captain_id'].astype('int')
)

daily_incentive_df['daily_burn_per_ride'] = (
    daily_incentive_df['daily_incentive_amt'].astype('int') /
    daily_incentive_df['dropped_order'].astype('int')
)

daily_avg_incentive_df = compute_aggregated_metrics(
    captain_daily_agg_df=daily_incentive_df,
    grouping_var=grouping_vars,
    agg_fields=['daily_burn_per_rider', 'daily_burn_per_ride'],
    agg_funcs=['mean', 'mean'],
)


net_rider_df = compute_aggregated_metrics(
    captain_daily_agg_df=rider_seg_exp_daily_df,
    grouping_var=grouping_vars,
    agg_fields=['captain_id'],
    agg_funcs=['nunique'],
)


daily_rpr_df = compute_aggregated_metrics(
    captain_daily_agg_df=rider_seg_exp_daily_df,
    grouping_var=['city_name', 'week_num', 'captain_id',
                  'poc_segment', 'experiment_group'],
    agg_fields=['dropped_order'],
    agg_funcs=['mean'],
)

daily_avg_rpr_df = compute_aggregated_metrics(
    captain_daily_agg_df=rider_seg_exp_daily_df,
    grouping_var=grouping_vars,
    agg_fields=['dropped_order'],
    agg_funcs=['mean'],
)
daily_avg_rpr_df.rename(columns={'dropped_order': 'daily_avg_rpr'}, inplace=True)
selector_size_df = compute_aggregated_metrics(
    captain_daily_agg_df=selectors_df,
    grouping_var=['poc_segment', 'experiment_group'],
    agg_fields=['rider'],
    agg_funcs=['nunique'],
)
selector_size_df.rename(columns={'rider': 'selector_size'}, inplace=True)

final_metric_df = (
    net_rider_df
    .merge(
        selector_size_df,
        on=['poc_segment', 'experiment_group']
    )
    .merge(daily_seg_metric_sum_df,on=grouping_vars,)
    .merge(daily_avg_incentive_df, on=grouping_vars)
    .merge(daily_avg_rpr_df, on=grouping_vars)
)

final_metric_df


  city_name  order_date                captain_id  dropped_order  \
0    Mumbai  2022-06-06  612c7eb334b93de406358227              2   
1    Mumbai  2022-06-06  610a46c780f61d07762cc5d4             20   
2    Mumbai  2022-06-06  62971adfa81bc548e659e1f4             11   
3    Mumbai  2022-06-06  5e728b45f49633436e2fa906              6   
4    Mumbai  2022-06-06  6234b3da6b6b643417706f1b             10   

   morning_order  afternoon_order  evening_night_order  mid_night_order  \
0              0                0                    0                2   
1              0                7                    8                5   
2              2                3                    3                3   
3              0                4                    2                0   
4              0                4                    5                1   

        date   logindate  login_hours  morning_duration  afternoon_duration  \
0 2022-06-06  2022-06-06     1.244167          0.000000      

,city_name,week_num,poc_segment,experiment_group,captain_id,selector_size,dropped_order,morning_order,afternoon_order,evening_night_order,mid_night_order,morning_duration,afternoon_duration,evening_duration,daily_incentive_amt,weekly_incentive_amt,daily_burn_per_rider,daily_burn_per_ride,daily_avg_rpr
0,Mumbai,2022-23,HP_D_HO,control_ride_group,190,304,2403,538,572,1136,157,359.228056,522.410833,766.145556,19810.0,1130.0,50.714440,8.237227,6.145780
1,Mumbai,2022-23,HP_D_HO,test_poc_segment,174,306,2351,573,466,1239,73,358.323611,464.874167,791.358889,11520.0,2180.0,31.846840,4.893603,6.494475
2,Mumbai,2022-23,HP_D_MO,control_ride_group,15,23,205,96,39,65,5,49.967222,26.953333,49.839444,2010.0,440.0,69.519481,9.568013,7.321429
3,Mumbai,2022-23,HP_D_MO,test_poc_segment,13,23,150,48,19,71,12,32.296667,18.943333,42.395833,1070.0,0.0,41.888889,6.905310,6.000000
4,Mumbai,2022-23,HP_D_UHO,control_ride_group,223,392,3387,755,899,1601,132,423.277222,752.500833,988.822222,30025.0,2910.0,63.442581,8.847660,7.160677
5,Mumbai,2022-23,HP_D_UHO,test_poc_segment,224,393,3415,818,929,1514,154,488.476111,787.748056,953.593889,20660.0,3810.0,43.444878,6.031799,7.174370
6,Mumbai,2022-23,LP_D_HO,control_ride_group,54,126,268,56,70,130,12,50.225278,111.675278,155.657500,1925.0,0.0,20.633180,7.010909,2.945055
7,Mumbai,2022-23,LP_D_HO,test_poc_segment,55,128,315,61,77,162,15,42.102500,108.495000,152.604167,2550.0,330.0,27.483245,8.129064,3.351064
8,Mumbai,2022-23,LP_D_LO,control_ride_group,104,273,621,130,138,305,48,96.843333,127.978333,254.166389,5420.0,1280.0,31.688596,8.702160,3.631579
9,Mumbai,2022-23,LP_D_LO,test_poc_segment,97,274,453,88,74,252,39,55.530556,99.444722,220.583056,3230.0,530.0,21.126543,7.102999,2.980263


### statistical significance test

In [9]:
def get_statistical_significance(
        df_for_stats, significance_test_on,
        experiment_grp_name, ctrl_grp_name,
        test_grp_name, two_sided_test=True):

    control_group = (
        df_for_stats[
            df_for_stats[experiment_grp_name] == ctrl_grp_name
        ][significance_test_on]
    )

    test_group = (
        df_for_stats[
            df_for_stats[experiment_grp_name] == test_grp_name
        ][significance_test_on]
    )

    if two_sided_test == False:
        test_output = stats.ttest_ind(control_group, test_group, alternative='less')
        alpha = 0.10
    else:
        test_output = stats.ttest_ind(control_group, test_group)
        alpha = 0.05

    t_calculated = test_output[0]
    p_val = test_output[1]

    if p_val > alpha:

        print(
            """
            As the p-value {:.4f} is greater than the thersold limit {},
            we fail to reject null hypothesis i.e. 
            There is no differece in the mean between Test and Group
            basis {} metric
            """.format(p_val, alpha, significance_test_on)
        )
    else:
        print(
            """
            As the p-value {:.4f} is greater than the thersold limit {},
            we fail to accept null hypothesis i.e. 
            There is a significant differece in the population mean between Test and Group
            basis {} metric
            """.format(p_val, alpha, significance_test_on)
        )
    
    return p_val


In [10]:

df_for_stats = daily_rpr_df.copy()
df_for_stats.rename(columns={'dropped_order': 'daily_rpr'}, inplace=True)
df_for_stats.dropna(inplace=True)
# print(df_for_stats.head())
significance_test_on = ['daily_rpr', 'login_hours', 'dropped_order', 'burn_per_ride']
test_grp_name = 'test_poc'
control_grp_name = 'control_ride_group'
experiment_col_name = 'experiment_group'
get_statistical_significance(
    df_for_stats=df_for_stats,
    significance_test_on=significance_test_on[0],
    experiment_grp_name=experiment_col_name,
    ctrl_grp_name=control_grp_name,
    test_grp_name=test_grp_name,
    two_sided_test=False
)



            As the p-value nan is greater than the thersold limit 0.1,
            we fail to accept null hypothesis i.e. 
            There is a significant differece in the population mean between Test and Group
            basis daily_rpr metric
            


nan

In [115]:
segment_list = df_for_stats[segment_col_name].dropna().unique()
significance_df = pd.DataFrame()
for segment in segment_list:
    print('Evaluation for segment \t: {}'.format(segment))
    temp_df = (
        df_for_stats[df_for_stats[segment_col_name] == segment]
    )
    temp_df.dropna(inplace=True)
    p_val = get_statistical_significance(
        df_for_stats=temp_df,
        significance_test_on=significance_test_on[0],
        experiment_grp_name=experiment_col_name,
        ctrl_grp_name=control_grp_name,
        test_grp_name=test_grp_name,
        two_sided_test=False
    )
    print(p_val)

    output_df =pd.DataFrame(data=[[segment, p_val]], columns=[['segment', 'p_value']])
    significance_df = pd.concat(
        [significance_df, output_df]
    )








Evaluation for segment 	: MP_D_MO

            As the p-value nan is greater than the thersold limit 0.1,
            we fail to accept null hypothesis i.e. 
            There is a significant differece in the population mean between Test and Group
            basis daily_rpr metric
            
nan
Evaluation for segment 	: LP_D_LO

            As the p-value nan is greater than the thersold limit 0.1,
            we fail to accept null hypothesis i.e. 
            There is a significant differece in the population mean between Test and Group
            basis daily_rpr metric
            
nan
Evaluation for segment 	: UHP_D_UHO

            As the p-value nan is greater than the thersold limit 0.1,
            we fail to accept null hypothesis i.e. 
            There is a significant differece in the population mean between Test and Group
            basis daily_rpr metric
            
nan
Evaluation for segment 	: LP_D_MO

            As the p-value nan is greater than the thersold 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [372]:
output = selectors_df.groupby(['poc_segment', 'experiment_group'])['rider'].count().reset_index()

In [233]:
# read last week
experiment_name = 'selectors_dormant'

experiment_selectors_file = (
    '{city}_{experiment_name}_wk_{week_num}.csv'
    .format(city=city, experiment_name = experiment_name, week_num=perf_wk)
)

print("reading last week {city} selector ...".format(city=city))
selectors_df = pd.read_csv(experiment_selectors_file)
selectors_df.head()

reading last week Hyderabad selector ...


,mobilenumber,simplified_segment,cohort
0,8897745098,HP_D_HO,dormant_control
1,9392474250,HP_D_HO,dormant_control
2,9908336595,HP_D_HO,dormant_control
3,7993974585,HP_D_HO,dormant_control
4,8897077231,HP_D_HO,dormant_control


In [90]:
# selectors_df.head()
selectors_df_holdout = pd.read_clipboard()

selector_holdout_req = pd.DataFrame()
for col in selectors_df_holdout.columns:
    temp_df = selectors_df_holdout[:][[col]]
    temp_df.dropna(inplace=True)
    temp_df = temp_df.astype('int')
    temp_df.columns = ['mobilenumber']
    temp_df['simplified_segment'] = col
    temp_df['cohort'] = 'dormant_holdout'
    selector_holdout_req = pd.concat([selector_holdout_req, temp_df])


In [89]:
selectors_df_test = pd.read_clipboard()

selector_test_req = pd.DataFrame()
for col in selectors_df_test.columns:
    temp_df = selectors_df_test[:][[col]]
    temp_df.dropna(inplace=True)
    temp_df = temp_df.astype('int')
    temp_df.columns = ['mobilenumber']
    temp_df['simplified_segment'] = col
    temp_df['cohort'] = 'dormant_test'
    selector_test_req = pd.concat([selector_test_req, temp_df])


In [93]:
final_selectors = pd.concat([selector_holdout_req, selector_test_req])
final_selectors.to_csv(experiment_selectors_file, index=False)


In [83]:
analysis_agg_df.to_clipboard()


In [6]:
def get_wow_retention(data, rider_col_name, date_col_name):
    """
    :param data: requires pandas dataframe with rider and ride date as the column
    :param return: week on week retention dataframe
    """

    data['year'] = (
        pd.to_datetime(data[date_col_name])
        .dt.isocalendar().year
    )

    data['week'] = (
        pd.to_datetime(data[date_col_name])
        .dt.isocalendar().week
    )

    data['week'] = data.week.map("{:02}".format)

    data['week'] = (
        data['year'].astype('str') +
        '-' +
        data['week'].astype('str')
    )

    weekly_agg_df = (
        data[[rider_col_name, 'week']]
        .groupby([rider_col_name, 'week'])
        .count()
        .reset_index()
    )

    week_list = weekly_agg_df['week'].unique()
    week_list.sort()
    temp_df = pd.DataFrame()

    for week_one, week_two in zip(week_list[:-1], week_list[1:]):

        week_n1_df = (
            weekly_agg_df[
                weekly_agg_df['week'] == week_one
            ]
            .rename(
                columns={
                    'week': 'week_n1',
                    rider_col_name: 'rider_wk_n1'
                }
            )
        )

        week_n2_df = (
            weekly_agg_df[
                weekly_agg_df['week'] == week_two
            ]
            .rename(
                columns={
                    'week': 'week_n2',
                    rider_col_name: 'rider_wk_n2'
                }
            )
        )

        wow_df = (
            pd.merge(
                left=week_n1_df,
                right=week_n2_df,
                left_on='rider_wk_n1',
                right_on='rider_wk_n2',
                how='left'
            )
        )

        temp_df = pd.concat([temp_df, wow_df])

    retention_view_df = (
        temp_df
        .groupby(['week_n1'])
        .agg(
            week_in=('rider_wk_n1', 'nunique'),
            week_next=('rider_wk_n2', 'nunique')
        )
        .reset_index()
        .rename(columns={'week_n1': 'week'})
    )

    return retention_view_df


def get_weekly_cohort_retention(data, date_field_name, user_id_col_name):
    """
    :param data: user level daily dataframe
    :param date_field_name: date column name in dataframe
    :param user_id_col_name: user id column name in dataframe
    :return: a dataframe giving WoW basis user turn up
    """

    data[date_field_name] = pd.to_datetime(data[date_field_name])
    
    min_ride_date_df = (
        data
        .groupby([user_id_col_name])
        .agg(min_ride_date=(date_field_name, min))
        .reset_index()
    )
    
    min_ride_date_df["week"] = (
        min_ride_date_df["min_ride_date"]
        .dt.isocalendar().week
    )
    min_ride_date_df = min_ride_date_df.drop("min_ride_date", axis=1)
    number_of_weeks = min_ride_date_df["week"].nunique()
    
    data["week"] = (
        data[date_field_name]
        .dt.isocalendar().week
    )
    
    weekly_users_df = (
        data
        .groupby(["week", user_id_col_name])
        .agg(occurence=(user_id_col_name, "count"))
        .reset_index()
    )

    weekly_users_df = weekly_users_df.drop("occurence", axis=1)
    

    min_week_num = weekly_users_df["week"].min()
    for val in range(0, number_of_weeks):
        temp_df = (
            min_ride_date_df[
                min_ride_date_df["week"] == min_week_num + val
            ]
        )

        new_df = pd.merge(
            weekly_users_df, temp_df,
            left_on=user_id_col_name,
            right_on=user_id_col_name,
            suffixes=("", '-' + str(min_week_num + val)),
            how="left"
        )
        weekly_users_df = new_df.copy()


    week_name_list = ["week-" + str(num) for num in range(min_week_num, min_week_num+number_of_weeks+1)]

    week_num_list = list(range(min_week_num, min_week_num+number_of_weeks+1))
    cohort_df = weekly_users_df.drop(user_id_col_name, axis=1)
    cohort_df = (
        cohort_df[
            cohort_df["week"] <= min_week_num + number_of_weeks-1
        ]
    )
    
    req_df = (
        cohort_df
        .groupby("week")
        .agg("count")
        .reset_index()
    )

    week_dict = dict(zip(week_num_list, week_name_list))
    req_df = (
        req_df
        .replace({"week": week_dict})
        .reset_index(drop=True)
        .set_index("week")
    )
    req_df = req_df.transpose()
    # cols = list(req_df.columns)
    # req_df = req_df[cols].div(req_df[cols].max(axis=0)).multiply(100)

    return req_df


In [7]:
week_7_df = pd.read_csv('Jaipur_captain_rides_df_wk_07.csv')
week_8_df = pd.read_csv('Jaipur_captain_rides_df_wk_08.csv')
week_9_df = pd.read_csv('Jaipur_captain_rides_df_wk_09.csv')
week_10_df = pd.read_csv('Jaipur_captain_rides_df_wk_10.csv')
week_11_df = pd.read_csv('Jaipur_captain_rides_df_wk_11.csv')

final_df = pd.concat(
    [
        week_7_df, week_8_df,
        week_9_df, week_10_df,
        week_11_df
    ]
)

print(final_df.head())

latest_selector_df = pd.read_csv('jaipur_iallocator_experiment_selectors_wk_09.csv')
print(latest_selector_df.head())


  city_name  order_date                captain_id  dropped_order
0    Jaipur  2022-02-14  5f4bcb0dbe98be0219371fe1             11
1    Jaipur  2022-02-14  5ce21f7f377155163cfefd65              4
2    Jaipur  2022-02-14  5d42434667ecd618f9ee2a41              7
3    Jaipur  2022-02-14  5d4134659f859318d1ef554f              9
4    Jaipur  2022-02-14  6197c5ff3f25c33a61e1a94c              3
   Unnamed: 0                     rider  mobilenumber simplified_segment  \
0           0  611b40028902097aa651d304    9610122951            HP_D_HO   
1           1  5f7c17617d99a7fdaa5b47fc    8949608537            HP_D_HO   
2           2  61768f51aec47fa49f978c33    9783183771            HP_D_HO   
3           3  60753d7879860e1a27b38986    7414870967            HP_D_HO   
4           4  5e200a199a0d23428ae4150a    8741996071            HP_D_HO   

       cohort    ride_segment  \
0  control_h1  (-0.001, 20.0]   
1  control_h1  (-0.001, 20.0]   
2  control_h1  (-0.001, 20.0]   
3  control_h1  (-0.00

In [8]:
# city retention number
req_retention_df = (
    final_df
    .merge(
        latest_selector_df,
        left_on=['captain_id'],
        right_on=['rider'],
        how='inner'
    )
    .reset_index(drop=True)
)
req_retention_df.head()


,city_name,order_date,captain_id,dropped_order,Unnamed: 0,rider,mobilenumber,simplified_segment,cohort,ride_segment,strat_buckets,shift,mobilenumber_prev
0,Jaipur,2022-02-14,5f4bcb0dbe98be0219371fe1,11,2355,5f4bcb0dbe98be0219371fe1,7023806594,MP_D_LO,control_h1,"(20.0, 40.0]","MP_D_LO_(20.0, 40.0]_(11.667, 277.0]_(18.333, ...",JAI_CM,NaN
1,Jaipur,2022-02-16,5f4bcb0dbe98be0219371fe1,3,2355,5f4bcb0dbe98be0219371fe1,7023806594,MP_D_LO,control_h1,"(20.0, 40.0]","MP_D_LO_(20.0, 40.0]_(11.667, 277.0]_(18.333, ...",JAI_CM,NaN
2,Jaipur,2022-02-20,5f4bcb0dbe98be0219371fe1,5,2355,5f4bcb0dbe98be0219371fe1,7023806594,MP_D_LO,control_h1,"(20.0, 40.0]","MP_D_LO_(20.0, 40.0]_(11.667, 277.0]_(18.333, ...",JAI_CM,NaN
3,Jaipur,2022-02-23,5f4bcb0dbe98be0219371fe1,9,2355,5f4bcb0dbe98be0219371fe1,7023806594,MP_D_LO,control_h1,"(20.0, 40.0]","MP_D_LO_(20.0, 40.0]_(11.667, 277.0]_(18.333, ...",JAI_CM,NaN
4,Jaipur,2022-02-25,5f4bcb0dbe98be0219371fe1,5,2355,5f4bcb0dbe98be0219371fe1,7023806594,MP_D_LO,control_h1,"(20.0, 40.0]","MP_D_LO_(20.0, 40.0]_(11.667, 277.0]_(18.333, ...",JAI_CM,NaN


In [9]:
expmnt_wow_reten_df = pd.DataFrame()

for val in req_retention_df.cohort.unique():
    temp_df = req_retention_df[
        req_retention_df['cohort'] == val
    ].reset_index()

    retention_df = get_wow_retention(
        data=temp_df,
        rider_col_name='captain_id',
        date_col_name='order_date'
    )
    
    retention_df.loc[:, 'experiment_name'] = val

    expmnt_wow_reten_df = pd.concat(
        [expmnt_wow_reten_df, retention_df]
    )

expmnt_wow_reten_df


,week,week_in,week_next,experiment_name
0,2022-07,3028,2421,control_h1
1,2022-08,2885,2215,control_h1
2,2022-09,2510,2060,control_h1
3,2022-10,2460,1871,control_h1
0,2022-07,3156,2521,test_h2
1,2022-08,3023,2328,test_h2
2,2022-09,2644,2151,test_h2
3,2022-10,2562,1939,test_h2


In [10]:

expmnt_seg_wow_reten_df = pd.DataFrame()
cohort_group_list = req_retention_df.cohort.unique()
segment_list = req_retention_df.simplified_segment.unique()
for group in cohort_group_list:
    for segment in segment_list:
        temp_df = req_retention_df[
            (req_retention_df['cohort'] == group) &
            (req_retention_df['simplified_segment'] == segment)
        ].reset_index()

        retention_df = get_wow_retention(
            data=temp_df,
            rider_col_name='captain_id',
            date_col_name='order_date'
        )

        retention_df.loc[:, 'experiment_name'] = group
        retention_df.loc[:, 'segment'] = segment

        expmnt_seg_wow_reten_df = pd.concat(
            [expmnt_seg_wow_reten_df, retention_df]
        )

expmnt_seg_wow_reten_df.head()


,week,week_in,week_next,experiment_name,segment
0,2022-07,22,17,control_h1,MP_D_LO
1,2022-08,21,16,control_h1,MP_D_LO
2,2022-09,19,16,control_h1,MP_D_LO
3,2022-10,21,17,control_h1,MP_D_LO
0,2022-07,411,331,control_h1,LP_D_MO


In [34]:
expmnt_seg_wow_reten_df.to_clipboard()


In [13]:
## get cohort retention for each group
experiment_groups = req_retention_df['cohort'].unique()
city_cohort_retention_df = pd.DataFrame()
seg_cohort_retention_df = pd.DataFrame()
segment_list = req_retention_df['simplified_segment'].unique()
for exp_grp_name in experiment_groups:
    for segment in segment_list:    
        temp_df = (
            req_retention_df[
                (req_retention_df['cohort'] == exp_grp_name) &
                (req_retention_df['simplified_segment'] == segment)
            ]
        ).reset_index()

        temp_retention_df = get_weekly_cohort_retention(
            temp_df, 'order_date', 'captain_id'
        )
        temp_retention_df.insert(0, 'experiment_group', exp_grp_name)
        temp_retention_df.insert(1, 'segment', segment)

        seg_cohort_retention_df = pd.concat(
            [seg_cohort_retention_df, temp_retention_df]
        )

for exp_grp_name in experiment_groups:
    temp_df = (
        req_retention_df[
            (req_retention_df['cohort'] == exp_grp_name)
        ]
    ).reset_index()
    temp_retention_df = get_weekly_cohort_retention(
    temp_df, 'order_date', 'captain_id'
    )
    temp_retention_df.insert(0, 'experiment_group', exp_grp_name)
    temp_retention_df.insert(1, 'segment', 'city')
    city_cohort_retention_df = pd.concat(
        [city_cohort_retention_df, temp_retention_df]
    )







In [14]:
# seg_cohort_retention_df.to_clipboard()
city_cohort_retention_df


week,experiment_group,segment,week-7,week-8,week-9,week-10,week-11
week-7,control_h1,city,3028,2421,2122,2040,1780
week-8,control_h1,city,0,464,271,253,210
week-9,control_h1,city,0,0,117,83,64
week-10,control_h1,city,0,0,0,84,44
week-11,control_h1,city,0,0,0,0,31
week-7,test_h2,city,3156,2521,2204,2162,1847
week-8,test_h2,city,0,502,308,276,240
week-9,test_h2,city,0,0,132,69,56
week-10,test_h2,city,0,0,0,55,30
week-11,test_h2,city,0,0,0,0,31


In [15]:
city_cohort_retention_df.to_clipboard()


### Generate Selectors Framework


#### prerequisite
- period to consider 28 days (4 rolling weeks)
- segment to consider latest
- order, login hours, incentive, total earnings


In [1]:
import os
file = os.listdir("/Users/rapido/Dharmendra/Photos/finalized_pics")


In [171]:
1410000 * .25

352500.0

In [95]:
get_start_end_date_from_week(22, 2022)

('2022-05-30', '2022-06-05')

In [96]:
test  =pd.read_clipboard()

In [98]:
pd.DataFrame(test.columns)

,0
0,city
1,rider
2,poc_segment
3,mobilenumber
4,ride_group
5,daily_incentive_bucket
6,daily_earnings_bucket
7,daily_lh_bucket
8,weekly_incentive_bucket
9,experiment_group
